## This notebook aims to merge all the ETF data into one csv file

In [22]:
import pandas as pd

Add some ETF data to the etf folder. Make sure that you choose the correct dates. Then add its name into the array below, and run this script to get a brand new csv with data from all the mentioned csv's. Also remember that the prices must be in USD.

In [23]:
'''
file_names = [
    'AGGG ETF Stock Price History.csv',
    'FXI ETF Stock Price History.csv',
    'HYG ETF Stock Price History.csv',
    'S&P 500 Historical Data.csv'
]
'''

file_names = [
    'ACWI.World.Equities.csv',
    'AGGG.World.Debt.csv',
    'AHYG.Asia.Debt.csv',
    'BWX.World.Debt.csv',
    'EDIV.EmergingMarkets.Equities.csv',
    'EEM.EmergingMarkets.Equities.csv',
    'EFA.World.Equities.csv',
    'EFAS.World.Dividend.Equities.csv',
    'EMB.EmergingMarkets.Debt.csv',
    'EPP.Pacific_ex_Japan.Equities.csv',
    'EWC.Canada.Equities.csv',
    'EWG.Europe.Equities.csv',
    'EWJ.Japan.Equities.csv',
    'EWU.Europe.Equities.csv',
    'EWZ.Brazil.Equities.csv',
    'FXI.China.Equities.csv',
    'GHYG.World.Debt.csv',
    'GLD.World.Commodities.csv',
    'GOVT.US.Debt.csv',
    'HYG.US.Debt.csv',
    'IHYG.Europe.Debt.csv',
    'IGOV.World.Debt.csv',
    'INDA.India.Equities.csv',
    'IWM.US.Equities.csv',
    'IEV.Europe.Equities.csv',
    'IEF.US.Debt.csv',
    'JNK.US.Debt.csv',
    'LQD.US.Debt.csv',
    'PSP.World.PrivateEquity.csv',
    'REET.World.Alternative.csv',
    'SAUS.Australia.Equities.csv',
    'SDY.US.Dividend.Equities.csv',
    'SPY.US.Equities.csv',
    'TLT.US.Debt.csv',
    'URTH.World.Equities.csv',
    'VEU.World.Equities.csv',
    'VTI.US.Equities.csv',
    'VWOB.EmergingMarkets.Debt.csv',
]


In [24]:
merged_df = pd.DataFrame()

In [25]:
df = pd.read_csv('etf_csv/ACWI.World.Equities.csv')

print(df.head())

  Exchange Date   Close      %Chg    Open      Low     High   Volume  \
0    2024-10-10  119.41 -0.001004  119.32  118.950  119.595   653784   
1    2024-10-09  119.53  0.004116  118.70  118.610  119.640  1163028   
2    2024-10-08  119.04  0.002611  118.72  118.540  119.140  1346099   
3    2024-10-07  118.73 -0.006277  119.35  118.395  119.380  2220201   
4    2024-10-04  119.48  0.008440  119.24  118.630  119.575   907360   

   Turnover - USD  
0      78022416.0  
1     138805055.0  
2     159973099.0  
3     263942542.0  
4     108049421.0  


In [26]:
''' make a df for all the csv files stored in etf_csv, removing every column except
 'Exchange Date' and 'Close'. Then merge them into one df on 'Exchange Date'. The 
  'Close' column for each df, should be renamed to the ETF name . '''

for file in file_names:
    df = pd.read_csv('etf_csv/' + file)
    df = df[['Exchange Date', 'Close']]

    filename = file.rsplit('.', 1)[0]
    
    df.rename(columns={'Close': filename}, inplace=True)
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='Exchange Date', how='outer')
        merged_df.sort_values(by='Exchange Date', ascending=False, inplace=True)


print(merged_df.head())

     Exchange Date  ACWI.World.Equities  AGGG.World.Debt  AHYG.Asia.Debt  \
3308    2024-10-10               119.41           4.3990            6.70   
3307    2024-10-09               119.53           4.3965            6.68   
3306    2024-10-08               119.04           4.4055            6.68   
3305    2024-10-07               118.73           4.4050            6.69   
3304    2024-10-04               119.48           4.4075            6.69   

      BWX.World.Debt  EDIV.EmergingMarkets.Equities  \
3308          22.500                          37.05   
3307          22.515                          36.94   
3306          22.655                          37.21   
3305          22.655                          37.96   
3304          22.700                          37.56   

      EEM.EmergingMarkets.Equities  EFA.World.Equities  \
3308                         46.03               81.88   
3307                         45.94               81.98   
3306                         46.19    

Setting the date as the index

In [27]:
merged_df.to_csv('asset_classes.csv', index=False)

Normalize it to get comparable plots of return over time. 

In [28]:
'''normalized_df = merged_df / df.iloc[0] * 100

normalized_df.reset_index(inplace=True)
''' 

merged_df['Exchange Date'] = pd.to_datetime(merged_df['Exchange Date'])

# Ensure all columns except 'Exchange Date' are numeric, coercing errors to NaN if necessary
merged_df.iloc[:, 1:] = merged_df.iloc[:, 1:].apply(
    pd.to_numeric, errors='coerce')

# Now, normalize based on the last row (oldest date)
normalized_df = merged_df.iloc[:, 1:] / merged_df.iloc[-1, 1:] * 100

# Add the 'Exchange Date' column back
normalized_df['Exchange Date'] = merged_df['Exchange Date']

# Reorder columns to bring 'Exchange Date' back to the front
normalized_df = normalized_df[['Exchange Date'] +
                              list(normalized_df.columns[:-1])]

In [29]:
normalized_df.head()

,Exchange Date,ACWI.World.Equities,AGGG.World.Debt,AHYG.Asia.Debt,BWX.World.Debt,EDIV.EmergingMarkets.Equities,EEM.EmergingMarkets.Equities,EFA.World.Equities,EFAS.World.Dividend.Equities,EMB.EmergingMarkets.Debt,...,PSP.World.PrivateEquity,REET.World.Alternative,SAUS.Australia.Equities,SDY.US.Dividend.Equities,SPY.US.Equities,TLT.US.Debt,URTH.World.Equities,VEU.World.Equities,VTI.US.Equities,VWOB.EmergingMarkets.Debt
3308,2024-10-10,277.697674,NaN,NaN,NaN,76.899128,117.359371,163.465762,NaN,84.964421,...,164.107579,NaN,207.469226,257.607497,446.162782,79.513185,NaN,153.787129,428.996676,NaN
3307,2024-10-09,277.976744,NaN,NaN,NaN,76.670818,117.129905,163.665402,NaN,85.10304,...,164.547677,NaN,207.735087,258.379272,446.944939,79.825896,NaN,153.910891,429.888184,NaN
3306,2024-10-08,276.837209,NaN,NaN,NaN,77.231216,117.767312,163.625474,NaN,85.24166,...,164.488753,NaN,207.537043,257.056229,443.870518,80.282285,NaN,154.158416,426.911454,NaN
3305,2024-10-07,276.116279,NaN,NaN,NaN,78.787879,120.750376,163.885007,NaN,85.10304,...,164.425428,NaN,209.438718,256.890849,439.711918,80.147059,NaN,155.470297,423.345422,NaN
3304,2024-10-04,277.860465,NaN,NaN,NaN,77.957659,119.756022,164.803354,NaN,85.491175,...,166.607335,NaN,209.202997,258.838662,443.72338,80.755578,NaN,155.767327,427.24388,NaN


In [30]:
normalized_df.to_csv('normalized_asset_classes.csv', index=False)

In [31]:
''' 
Takes the merged_df and selects a selection of columns to include (and exclude the rest)
Then creates a new csv file with the selected columns
'''

selected_columns = [ 
    'Exchange Date',
    'ACWI.World.Equities',
    #'AGGG.World.Debt', not enough data
    'IGOV.World.Debt',
    'VWOB.EmergingMarkets.Debt',
    'VEU.World.Equities',
    'LQD.US.Debt',
    'SPY.US.Equities',
    'REET.World.Alternative',
    'PSP.World.PrivateEquity',
]

selected_df = merged_df[selected_columns]

selected_df.to_csv('selected_asset_classes.csv', index=False)
